In [1]:
# Importing Modules

In [4]:
import os
import snowflake.connector
import pandas as pd


In [5]:
try:
    # Connect to Snowflake using environment variables
    snowflake_conn = snowflake.connector.connect(
        user='SUMIT45',
        password='Mrsumit45@',
        account='oq86887.central-india.azure',
        warehouse="COMPUTE_WH",
        database="FINAL_PROJECT",
        schema='RAW',
        role = 'ACCOUNTADMIN'
    )

    # Print connection success message
    print("Connected to Snowflake successfully!")

    # Now, you can perform further operations with snowflake_conn
except snowflake.connector.errors.DatabaseError as e:
    # Print connection failure message
    print(f"Failed to connect to Snowflake: {e}")

Connected to Snowflake successfully!


## Ingest Into Snowflake

In [6]:
# Create staging_raw_data folder if it doesn't exist
if not os.path.exists("staging_raw_data"):
    print("No data to process. Exiting.")
    exit()

# Iterate over each CSV file in the staging_raw_data folder
for filename in os.listdir("staging_raw_data"):
    if filename.endswith(".csv"):
        # Extract table name from filename (remove .csv extension)
        table_name = os.path.splitext(filename)[0]
        
        # Read CSV file into DataFrame
        df = pd.read_csv(f"staging_raw_data/{filename}")
        
        # Replace NaN values with empty strings
        df = df.fillna('')
        
        # Convert all data to string
        df = df.astype(str)
        
        # Create table in Snowflake if it doesn't exist
        snowflake_cursor = snowflake_conn.cursor()
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
        for column in df.columns:
            create_table_query += f"{column} VARCHAR,"
        create_table_query = create_table_query[:-1] + ")"  # Remove trailing comma
        snowflake_cursor.execute(create_table_query)
        
        # Prepare INSERT INTO statement
        insert_query = f"INSERT INTO {table_name} VALUES ({','.join(['%s'] * len(df.columns))})"
        
        # Convert DataFrame to list of tuples (rows)
        rows = [tuple(row) for row in df.itertuples(index=False)]
        
        # Execute bulk insert
        snowflake_cursor.executemany(insert_query, rows)
        snowflake_cursor.close()
        
        print(f"Data from '{filename}' inserted into '{table_name}' table in Snowflake.")

# Commit the transaction
snowflake_conn.commit()

# Close Snowflake connection
snowflake_conn.close()

Data from 'Report.csv' inserted into 'Report' table in Snowflake.
Data from 'Subrole.csv' inserted into 'Subrole' table in Snowflake.
Data from 'TrainingAmount.csv' inserted into 'TrainingAmount' table in Snowflake.
Data from 'TrainingAssessment.csv' inserted into 'TrainingAssessment' table in Snowflake.
Data from 'TrainingCategory.csv' inserted into 'TrainingCategory' table in Snowflake.
Data from 'TrainingModule.csv' inserted into 'TrainingModule' table in Snowflake.
Data from 'TrainingPlan.csv' inserted into 'TrainingPlan' table in Snowflake.
Data from 'User.csv' inserted into 'User' table in Snowflake.
Data from 'UserTrainingProgress.csv' inserted into 'UserTrainingProgress' table in Snowflake.
